In [1]:
!pip install torch
!pip install torchvision

In [3]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
#Adam optimizer


In [4]:

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cpu


In [6]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [7]:
#Dataloader

#Path for training and testing directory , images are in the training and test sets
train_path= "C:/Users/Noah Lim Ren Dong/Desktop/Data/seg_train"
test_path= "C:/Users/Noah Lim Ren Dong/Desktop/Data/seg_test"

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [8]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [9]:
print(classes)

['Emergency', 'Empty', 'Normal']


In [10]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
 
    
        
        
        
    
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        

        #forward propagation
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            

         
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [11]:
model=ConvNet(num_classes=3).to(device)

In [12]:

optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [13]:
num_epochs=20

In [14]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [15]:
print(train_count,test_count)

601 74


In [16]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
  
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'topdown_Showcase_Latest_best_checkpoint_Emptyroom.model')
        best_accuracy=test_accuracy
    
    


C:\Users\Noah Lim Ren Dong\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 Train Loss: tensor(4.6076) Train Accuracy: 0.7437603993344426 Test Accuracy: 0.7972972972972973
Epoch: 1 Train Loss: tensor(1.2839) Train Accuracy: 0.9201331114808652 Test Accuracy: 0.8243243243243243
Epoch: 2 Train Loss: tensor(0.4333) Train Accuracy: 0.9733777038269551 Test Accuracy: 0.8648648648648649
Epoch: 3 Train Loss: tensor(0.2836) Train Accuracy: 0.9700499168053245 Test Accuracy: 1.0
Epoch: 4 Train Loss: tensor(0.2193) Train Accuracy: 0.9750415973377704 Test Accuracy: 0.9864864864864865
Epoch: 5 Train Loss: tensor(0.1673) Train Accuracy: 0.9883527454242929 Test Accuracy: 0.9459459459459459
Epoch: 6 Train Loss: tensor(0.1165) Train Accuracy: 0.9850249584026622 Test Accuracy: 0.9864864864864865
Epoch: 7 Train Loss: tensor(0.1591) Train Accuracy: 0.9833610648918469 Test Accuracy: 1.0
Epoch: 8 Train Loss: tensor(0.0010) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 9 Train Loss: tensor(1.3890e-05) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 10 Train Loss: tensor(0.0393)